## 使用微调后的 LLaMA2-7B 推理

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


model_dir = "models/llama-7-int4-dolly-20250823_162749"
 
# 加载基础LLM模型与分词器
model = AutoPeftModelForCausalLM.from_pretrained(
    model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
) 
tokenizer = AutoTokenizer.from_pretrained(model_dir)

/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from datasets import load_dataset 
from random import randrange
 
 
# 从hub加载数据集并得到一个样本
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
sample = dataset[randrange(len(dataset))]
 
prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 
 
### Input:
{sample['response']}
 
### Response:
"""
 
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample['instruction']}")

Prompt:
The stroller that's the best fit for you depends on what characteristics you value. Example characteristics can be: affordability, ease of pushing and steering, ease of folding, ability to grow with your family, aesthetics, weight, and availability of high-end features like a bassinet attachment or large storage basket.

If you care about affordability, ease of folding, and ease of pushing and steering, the Baby Jogger City Mini GT2 stroller is a great option. If you care about the ability of the stroller to growth with your family, ease of pushing and steering, and high-end features, the Uppababy Vista V2 stroller is another popular option. If you have twins, the Bugaboo Donkey 5 stroller could be a great fit with its side-by-side double configuration.

Generated instruction:
What are the best strollers to buy?

Ground truth:
What stroller should I buy for my newborn baby?
